In [ ]:
!pip install transformers
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('/content/drive/MyDrive/orcas_5000.tsv', sep='\t')


In [ ]:
df.columns =['Qid1', 'Qid2', 'Query', 'Did','Link','Level1','Level2']

In [ ]:
df.head()

,Qid1,Qid2,Query,Did,Link,Level1,Level2
0,14940104,7574702,symptoms of endometrial (uterine) cancer,D1279756,http://www.cancer.ca/en/cancer-information/can...,Abstain,Abstain
1,4342021,5289265,desert,D337616,https://www.worldatlas.com/articles/10-largest...,Abstain,Abstain
2,13767102,7787863,schwinn exercise bicycles,D2581278,http://www.schwinn.com/,Abstain,Abstain
3,2615076,12100561,car rentals kansas city international airport,D2663272,https://www.hertz.com/rentacar/location/united...,Abstain,Abstain
4,15835054,11406475,trout diet,D871646,http://www.streamexplorers.org/fish-facts/trou...,Abstain,Abstain


In [ ]:
df['final_label'] = df['Level1']

In [ ]:
df.head()

,Qid1,Qid2,Query,Did,Link,Level1,Level2,final_label
0,14940104,7574702,symptoms of endometrial (uterine) cancer,D1279756,http://www.cancer.ca/en/cancer-information/can...,Abstain,Abstain,Abstain
1,4342021,5289265,desert,D337616,https://www.worldatlas.com/articles/10-largest...,Abstain,Abstain,Abstain
2,13767102,7787863,schwinn exercise bicycles,D2581278,http://www.schwinn.com/,Abstain,Abstain,Abstain
3,2615076,12100561,car rentals kansas city international airport,D2663272,https://www.hertz.com/rentacar/location/united...,Abstain,Abstain,Abstain
4,15835054,11406475,trout diet,D871646,http://www.streamexplorers.org/fish-facts/trou...,Abstain,Abstain,Abstain


In [ ]:
df.loc[df['final_label']== "Abstain","final_label"] = df.loc[df['final_label']== "Abstain","Level2"]

In [ ]:
df.tail()

,Qid1,Qid2,Query,Did,Link,Level1,Level2,final_label
4994,14110477,6454105,simmons online msw,D1344743,https://www.onlinemswprograms.com/school-profi...,Transactional,Abstain,Transactional
4995,2061186,4980436,bmo maple ridge,D1134893,https://www.bmo.com/,Abstain,Abstain,Abstain
4996,14509183,5273460,sql server median,D2092249,http://blogs.lessthandot.com/index.php/DataMgm...,Abstain,Abstain,Abstain
4997,9024388,12142812,john brown college,D2220671,https://en.wikipedia.org/wiki/John_Brown_Unive...,Abstain,Factual,Factual
4998,7869533,12746216,how to get rid of lawn weeds,D1372551,https://www.familyhandyman.com/landscaping/law...,Abstain,Instrumental,Instrumental


In [ ]:
df.head()

,Qid1,Qid2,Query,Did,Link,Level1,Level2,final_label
0,14940104,7574702,symptoms of endometrial (uterine) cancer,D1279756,http://www.cancer.ca/en/cancer-information/can...,Abstain,Abstain,Abstain
1,4342021,5289265,desert,D337616,https://www.worldatlas.com/articles/10-largest...,Abstain,Abstain,Abstain
2,13767102,7787863,schwinn exercise bicycles,D2581278,http://www.schwinn.com/,Abstain,Abstain,Abstain
3,2615076,12100561,car rentals kansas city international airport,D2663272,https://www.hertz.com/rentacar/location/united...,Abstain,Abstain,Abstain
4,15835054,11406475,trout diet,D871646,http://www.streamexplorers.org/fish-facts/trou...,Abstain,Abstain,Abstain


In [ ]:
df['final_label'].value_counts()

Abstain          3156
Factual          1031
Instrumental      460
Navigational      218
Transactional     134
Name: final_label, dtype: int64

In [ ]:
possible_labels = df.final_label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Abstain': 0,
 'Factual': 2,
 'Instrumental': 1,
 'Navigational': 4,
 'Transactional': 3}

In [ ]:
df['label'] = df.final_label.replace(label_dict)

In [ ]:
df.tail()

,Qid1,Qid2,Query,Did,Link,Level1,Level2,final_label,label
4994,14110477,6454105,simmons online msw,D1344743,https://www.onlinemswprograms.com/school-profi...,Transactional,Abstain,Transactional,3
4995,2061186,4980436,bmo maple ridge,D1134893,https://www.bmo.com/,Abstain,Abstain,Abstain,0
4996,14509183,5273460,sql server median,D2092249,http://blogs.lessthandot.com/index.php/DataMgm...,Abstain,Abstain,Abstain,0
4997,9024388,12142812,john brown college,D2220671,https://en.wikipedia.org/wiki/John_Brown_Unive...,Abstain,Factual,Factual,2
4998,7869533,12746216,how to get rid of lawn weeds,D1372551,https://www.familyhandyman.com/landscaping/law...,Abstain,Instrumental,Instrumental,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
print(df['data_type'])

0       train
1         val
2       train
3       train
4         val
        ...  
4994    train
4995    train
4996    train
4997    train
4998    train
Name: data_type, Length: 4999, dtype: object


In [ ]:
df.groupby(['final_label', 'label', 'data_type']).count()

Qid1  Qid2  Query   Did  Link  Level1  Level2
final_label   label data_type                                               
Abstain       0     train      2683  2683   2683  2683  2683    2683    2683
                    val         473   473    473   473   473     473     473
Factual       2     train       876   876    876   876   876     876     876
                    val         155   155    155   155   155     155     155
Instrumental  1     train       391   391    391   391   391     391     391
                    val          69    69     69    69    69      69      69
Navigational  4     train       185   185    185   185   185     185     185
                    val          33    33     33    33    33      33      33
Transactional 3     train       114   114    114   114   114     114     114
                    val          20    20     20    20    20      20      20

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Query.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Query.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
print(input_ids_train)
attention_masks_train = encoded_data_train['attention_mask']
print(attention_masks_train)
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([[  101,  8030,  1997,  ...,     0,     0,     0],
        [  101,  8040,  2232,  ...,     0,     0,     0],
        [  101,  2482, 12635,  ...,     0,     0,     0],
        ...,
        [  101, 29296,  8241,  ...,     0,     0,     0],
        [  101,  2198,  2829,  ...,     0,     0,     0],
        [  101,  2129,  2000,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(4249, 750)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/133 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.9285075424757219
Validation loss: 0.6581868988772234
F1 Score (Weighted): 0.771196545535364


Epoch 2:   0%|          | 0/133 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5494778727678428
Validation loss: 0.4913693517446518
F1 Score (Weighted): 0.8612742380582966


...


Epoch 8
Training loss: 0.18315746153431728
Validation loss: 0.5089013275379936
F1 Score (Weighted): 0.8744531665950669


Epoch 9:   0%|          | 0/133 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.15932917601912094
Validation loss: 0.515838318814834
F1 Score (Weighted): 0.8745280513706345


Epoch 10:   0%|          | 0/133 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.15012594571519167
Validation loss: 0.5177310605843862
F1 Score (Weighted): 0.8770201137050253


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model' ))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)
f1_score_func(predictions, true_vals)

Class: Abstain
Accuracy: 449/473

Class: Instrumental
Accuracy: 47/69

Class: Factual
Accuracy: 107/155

Class: Transactional
Accuracy: 15/20

Class: Navigational
Accuracy: 33/33



0.8635598666108766

In [ ]:
model = model.to(device)

In [ ]:
inputs = tokenizer("make America great again", return_tensors="pt").to(device)
labels = torch.tensor([0]).unsqueeze(0).to(device)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits


In [ ]:
loss

tensor(3.0634, device='cuda:0', grad_fn=<NllLossBackward>)

In [ ]:
logits

tensor([[ 0.1312,  3.0549,  0.2360, -0.6342, -1.4111]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [ ]:
labels

tensor([[0]], device='cuda:0')

In [ ]:
labels

tensor([[0]], device='cuda:0')

In [ ]:
np.exp(logits.cpu().detach().numpy()) / np.sum(np.exp(logits.cpu().detach().numpy()))

array([[0.04672921, 0.8696422 , 0.05189618, 0.02173739, 0.00999507]],
      dtype=float32)

In [ ]:
9.9927390e-01

0.9992739

In [ ]:
def pred(test_data):
  l = []
  for query, label in test_data:
    inputs = tokenizer(query, return_tensors="pt").to(device)
    labels = torch.tensor([label_dict[label]]).unsqueeze(0).to(device)  # Batch size 1
    print(label_dict[label])
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    logits = outputs.logits
    sm = np.exp(logits.cpu().detach().numpy()) / np.sum(np.exp(logits.cpu().detach().numpy()))
    l.append((sm,loss))
  return l


In [ ]:
pr = pred([("medical assistance","Factual"),("what is my name","Factual")])
print(pr)
print(label_dict)

2
2
[(array([[0.9809124 , 0.00646646, 0.0076656 , 0.00320157, 0.00175402]],
      dtype=float32), tensor(4.8710, device='cuda:0', grad_fn=<NllLossBackward>)), (array([[0.00517581, 0.00366004, 0.98448956, 0.00285215, 0.00382242]],
      dtype=float32), tensor(0.0156, device='cuda:0', grad_fn=<NllLossBackward>))]
{'Abstain': 0, 'Instrumental': 1, 'Factual': 2, 'Transactional': 3, 'Navigational': 4}
